In [55]:
from nltk import WordPunctTokenizer
import os, pprint
from nltk import WordNetLemmatizer
from nltk import regexp_tokenize, sent_tokenize, word_tokenize
from nltk import PorterStemmer
from nltk.corpus import stopwords
from operator import itemgetter

In [56]:
stop_ws = set(stopwords.words('english'))

cwd = os.getcwd()
dataPath = "graphics_motorcycles/"

docs = {}

In [57]:
# def rename_documents(dataPath):
    
#     new_path = cwd+'/'+dataPath
#     every_path = list(os.walk(new_path))
    
#     total = 1
#     for i in range(0,len(every_path)):
#         dirPath, dirName, fileNames = every_path[i]
#         if(len(dirName) == 0):
#             for j in fileNames:
#                 single_doc_loc = dirPath + '/' + j
#                 os.rename(single_doc_loc, dirPath + '/' + str(total))
#                 total+=1
                

In [58]:
# rename_documents(dataPath)

In [59]:
def collect_documents(dataPath):
    global docs
    
    new_path = cwd+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc = []
                single_doc_loc = dirPath + '/' + j
                with open(single_doc_loc, 'rb') as f:
                    single_doc.append(str(f.read()))
                docs[j] = single_doc


In [60]:
collect_documents(dataPath)

In [61]:
def tokenizeDocument(document):
    """
        document : string
        
        0. Convert to lowercase
        1. Stop words removed
        2. Tokenize 
        3. Stemming
        4. Lemmatization
        5. Only words that starts with alphabet or digit. Front 0's removed.
    """
    ts = document.split('\\n')
    document = ' '.join(ts)
    ts = document.split('\t')

    document = ' '.join(ts)
  
    # Tokenization
    tokens = WordPunctTokenizer().tokenize(document)

    # lowercase
    tokens_lowercase = [ w.lower() for w in tokens]
    
    #Remove Stop words
    tokens_stop  = [ w for w in tokens_lowercase if(w not in stop_ws)] 
    
    # Stemming 
    tokens_stem = [ PorterStemmer().stem(w) for w in tokens_stop]   # .wes. we

    # Lemmatization
    updated_tokens = [ WordNetLemmatizer().lemmatize(w) for w in tokens_stem]
     
    final_tokens = []
    
    for updated_token in updated_tokens:
        if(updated_token[0].isalpha()) and (len(updated_token) > 1):
            final_tokens.append(updated_token)
        else:
            if(updated_token.isnumeric()):
                final_tokens.append(str(int(updated_token)))
            else:
                if(updated_token[0].isdigit()):
                    updated_token = updated_token.lstrip('0')
                    final_tokens.append(updated_token)
        
    
    final_tokens = final_tokens[1:]  # remove b

    # Unique only
    final_tokens = list(set(final_tokens))

    return final_tokens

In [62]:
def convertDocument(my_document):
    my_document = my_document[1:]
    all_sent = my_document.split('\\n')[:-1]
    
    final_doc = {}
    temp_doc= []
    tokenized_doc = []

    for s in all_sent:
        if(s!=''):         
            s = s.replace('\\t',' ')
            s = s.replace('(',' ')
            s = s.replace(')',' ')
            s = s.replace(';',' ')
            s = s.replace('"',' ')
            s = s.replace('!',' ')
            s = s.replace('<',' ')
            s = s.replace('>',' ')
            s = s.replace(',',' ')
#             s = s.replace('--',' ')
            s = [w.lower() for w in s]
            s = ''.join(s)
            temp_doc.append(word_tokenize(s))
    
    
    c = 0
    
    for i in temp_doc:
        flag = 0
        for j in i:
            if(j[0].isalpha() or j[0].isdigit()):
                if(j[-1] == '.' or j[-1] == ':'):
                    final_doc[c+1] = j[0:-1]
                    c+=1
                else:
                    final_doc[c+1] = j
                    c+=1
            
            if(j[0] == '.' and j[1:].isalpha()):
                    final_doc[c+1] = j
                    c+=1
    return final_doc

    

In [63]:
positional_index = {}  # keys = terms val= [freq, { key = docid, val = [freq, [pos]]}]

for k,v in docs.items():
    docs[k] = convertDocument(v[0])   # convert document string to dictionary with keys as a position

In [64]:
for k,v in docs.items():
    for pos,term in v.items():
        if(term in positional_index.keys()):
            positional_index[term][0]+=1    # term freq at upper level
            pos_iv = positional_index[term][1]

            if(k in pos_iv.keys()):
                pos_iv[k][0]+=1  # term freq at  lower level 
                pos_iv[k][1].append(pos)  # add the new position
            else:
                pos_iv[k] = [1, []]
                pos_iv[k][1].append(pos)
        else:
            positional_index[term] = [1,{}] # terms freq, docid's        
            pos_iv = positional_index[term][1]
            pos_iv[k] = [1, []]
            pos_iv[k][1].append(pos)


In [65]:

# pprint.pprint(positional_index)

In [66]:
# Store the index in a file
with open('positional-index.txt','w') as f:
    print(positional_index, file=f)

In [67]:
print(positional_index.keys())

dict_keys(['newsgroups', 'comp.graphics', 'path', 'cantaloupe.srv.cs.cmu.edu', 'magnesium.club.cc.cmu.edu', 'news.sei.cmu.edu', 'cis.ohio-state.edu', 'zaphod.mps.ohio-state.edu', 'caen', 'destroyer', 'gumby', 'calvin', 'ursa', 'eball12', 'from', 'ursa.calvin.edu', 'edward', 'ball', 'subject', 'ibm', 'writing', 'to', 'screen', 'memory', 'graphics', 'message-id', 'eball12.736125322', 'sender', 'news', 'calvin.edu', 'the', 'annoying', 'pseudo-user', 'nntp-posting-host', 'organization', 'college', 'date', 'thu', '29', 'apr', '1993', '23:15:22', 'gmt', 'lines', '27', 'can', 'anyone', 'give', 'me', 'information', 'or', 'lead', 'electronic', 'not', 'books', 'i', 'too', 'poor', 'regarding', 'programming', 'standard', 'modes', '320x200x4', 'and', '640x200x2', 'are', 'easy', 'enough', 'but', 'so', 'sure', 'about', 'rest', 'something', 'planes', 'ports', 'like', 'do', "n't", 'know', 'numbers', 'anything', 'for', '16', 'color', 'think', 'if', 'wrong', 'let', 'also', '320x200x256', 'is', 'just', 'o

In [68]:

def fun(k1_pos, k2_pos, d):
    for i in k1_pos:
        for j in k2_pos:
            if(j-i) == d:
                return True
            
    return False
    
def processPhraseQuery(k1, k2, d):
    
    mid_result = []
    
    # Get posting list of k1 and k2
    if k1 in positional_index.keys() and k2 in positional_index.keys():
        pl_k1 = positional_index[k1][1].keys() # doc ids which have k1
        pl_k2 = positional_index[k2][1].keys() # doc ids which have k2
        
        match_pl = list(set(pl_k1) & set(pl_k2))
        
        for match in match_pl:
            
            #find the matching positions
            k1_pos = positional_index[k1][1][match][1]
            k2_pos = positional_index[k2][1][match][1]
            
            if(fun(k1_pos, k2_pos, d)):
                mid_result.append(match)                      
            
    return mid_result      

In [69]:
query_result = []

st = str(input('Enter your query: '))
keywords = st.split(' ')                  

Enter your query: Actually any algorithm would be nice 


In [70]:
for i in range(0,len(keywords)):
    for j in range(i+1,len(keywords)):
        if(i!=j):
            query_result.append(processPhraseQuery(keywords[i], keywords[j], j-i))   
    

In [71]:
answer = set(query_result[0])
print(answer)
for k in query_result[1:]:
    answer = answer & set(k)
print('Documents found: ',answer)

set()
Documents found:  set()


In [49]:
print(query_result)

[[], ['684', '392', '1067', '1479', '146', '402', '178', '233', '671', '415', '828', '1284', '665', '1085'], ['197', '1067', '1830', '874', '178', '1113', '713', '415', '828', '574', '936', '684', '402', '365', '787', '1284', '778', '424', '1147', '233', '671', '655', '256'], ['1177', '1629', '333', '1503', '361', '514', '1103', '189', '438', '1712', '1164', '1006', '1686', '800', '1830', '510', '1850', '1188', '719', '1073', '157', '15', '1729', '1291', '1255', '1251', '766', '376', '352', '936', '1350', '1748', '95', '1831', '456', '728', '145', '1074', '691', '33', '1216', '240', '1155', '250', '319', '1220', '860', '811', '770', '1668', '1263', '1488', '144', '1706', '455', '1217', '621', '839', '1947', '1489', '423', '1654', '392', '778', '516', '1421', '919', '1021', '1354', '214', '1659', '1089', '354', '761', '276', '659', '977', '1724', '63', '1192', '1444', '310', '1471', '1771', '359', '1601', '1764', '583', '594', '1039', '191', '351', '1238', '1879', '1722', '1558', '1062'